Load libraries

In [2]:
#imports
from __future__ import print_function
import pyart
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import numpy as np
import h5py
from time import time
import os
from netCDF4 import num2date, date2num

In [5]:
def odim_to_nc(in_fn,out_fn,kurnell_flag):

    #read fn
    radar = pyart.aux_io.read_odim_h5(in_fn, file_field_names=True) 

    #update field names
    radar.fields['DBZH']['standard_name'] = 'Reflectivity'
    radar.fields['DBZH']['units'] = 'dBZ'
    radar.fields['DBZH']['long_name'] = 'Radar Reflectivity Factor'
    radar.fields['VRADH']['standard_name'] = 'Velocity'
    radar.fields['VRADH']['units'] = 'm/s'
    radar.fields['VRADH']['long_name'] = 'Radial Velocity of Scatterers'

    #create custom nyquist velocity field (adjusting for kurnell)
    hfile = h5py.File(in_fn, 'r')
    d1_how = hfile['dataset1']['how'].attrs
    NI = d1_how['NI']
    NI_vec = np.ones(radar.nsweeps)*NI
#    if kurnell_flag == 1:
#        NI_onsite = 27
#        NI_vec[1:] = NI_onsite
    new_dict = {'data': NI_vec, 'units': 'meters_per_second', 'meta_group':'instrument_parameters','comments':'Unambiguous velocity','long_name':'Nyquist Velocity'}
    radar.instrument_parameters = {'nyquist_velocity':new_dict}    

    #apply dealias filter
    gatefilter = pyart.correct.GateFilter(radar)
    gatefilter.exclude_masked('DBZH')
    corr_vel = pyart.correct.dealias_region_based(radar, vel_field='VRADH', gatefilter = gatefilter, nyquist_vel=NI_vec)
    radar.add_field('VRADH_corr', corr_vel, False)

    #write to nc
    pyart.io.write_cfradial(out_fn, radar)

In [6]:
in_dirr = '/run/media/meso/DATA/project_data/SydneyBushFire/71_odimh5/'
out_dirr = '/run/media/meso/DATA/project_data/SydneyBushFire/nick_cf_20131017/71_cfradial/'

fls = os.listdir(in_dirr)
fls.sort()

i = 1
for fl in fls:
    print('doing file ',i,' ',in_dirr+ fl)
    in_fn  = in_dirr+fl
    out_fn = out_dirr+fl+'.nc'
    odim_to_nc(in_fn,out_fn,0)
    i = i +1




doing file  1   /run/media/meso/DATA/project_data/SydneyBushFire/71_odimh5/02_20131017_000100.h5
doing file  2   /run/media/meso/DATA/project_data/SydneyBushFire/71_odimh5/02_20131017_000700.h5
doing file  3   /run/media/meso/DATA/project_data/SydneyBushFire/71_odimh5/02_20131017_001300.h5
doing file  4   /run/media/meso/DATA/project_data/SydneyBushFire/71_odimh5/02_20131017_001900.h5
doing file  5   /run/media/meso/DATA/project_data/SydneyBushFire/71_odimh5/02_20131017_002500.h5
doing file  6   /run/media/meso/DATA/project_data/SydneyBushFire/71_odimh5/02_20131017_003100.h5
doing file  7   /run/media/meso/DATA/project_data/SydneyBushFire/71_odimh5/02_20131017_003700.h5
doing file  8   /run/media/meso/DATA/project_data/SydneyBushFire/71_odimh5/02_20131017_004300.h5
doing file  9   /run/media/meso/DATA/project_data/SydneyBushFire/71_odimh5/02_20131017_004900.h5
doing file  10   /run/media/meso/DATA/project_data/SydneyBushFire/71_odimh5/02_20131017_005500.h5
doing file  11   /run/media/m